Create a Feature Layer for each combination of Map Display Geography (4) x Variables (7) wherein the attribute fields are 

  Like this for Households (for the other 6 variables substitute hh_ with: pop_, ijb_, ojb, rjb_, tjb_,  hji_)

TAZID, city, county, med_dist, small_dist, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Sml_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Med_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
CityID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50


### For the map:
<!-- 1. obtain taz 832 and 900 shapefile -->
<!-- 2. attributes we need:
  - total households
  - total population
  - total jobs
  - industrial jobs(5,10) 
  - retail jobs(1,9) 
  - office jobs(6,4,3)
  - typical jobs (sum of three types, calculated)
  - HJ Intensity (this is (HH * 1.8) + Jobs) -->
<!-- 3. get taz outout from 2019 REMM -->
<!-- 4. get Official TAZ output from 2015 REMM (confirm TAZ version) -->
<!-- 5. Generate folder structure -->
6. Apportion new taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
7. Apportion old taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
8. create shapefiles/feature classes for each attribute (see #2)

### For the chart:
1. create a json for each record using the following geographies:
  - taz
  - medium district
  - city
  - small district (later)



In [1]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [42]:
# import datasets
# parcels = r'.\\Inputs\\remm_base_year_20220513.gdb\\parcels_2019'
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"
taz900 = r".\Inputs\TAZ_900.shp"
cities = r".\Inputs\CityArea.shp"
# small_districts = ???
# regions_df = pd.DataFrame.spatial.from_featureclass(regions_shp)

taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)
taz900_sdf = pd.DataFrame.spatial.from_featureclass(taz900)

In [12]:
pm_folder = r".\New_SE_Data"
new_taz_se = glob.glob(os.path.join(pm_folder,'SE_WF_*.csv'))
len(new_taz_se)

32

In [5]:
old_se_folder = r".\Old_SE_Data"
old_taz_se = glob.glob(os.path.join(old_se_folder,'*_TAZ.csv'))
old_city_se = glob.glob(os.path.join(old_se_folder,'*_City_Area.csv'))

In [40]:
outputs = r'.\Outputs'

scratch = os.path.join(outputs, 'scratch.gdb')
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, 'scratch.gdb')

map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [34]:
base = taz832_sdf[['TAZID','SHAPE']].copy()
base.shape

(2881, 2)

In [22]:
# year = os.path.splitext(os.path.basename(new_taz_se[0]))[0].split('_')[-1]
# x = pd.read_csv(new_taz_se[0])
# x['OFFI'] = x['OFFI'] + x['GVED'] + x['HLTH']
# x['TPCL'] = x['OFFI'] + x['RETEMP'] + x['INDEMP']
# x['HJI'] = x['TOTHH']*1.8 + x['TOTEMP']
# x = x[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
# x.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
# x


,TAZID,HH_2019,POP_2019,EMP_2019,RTL_2019,IND_2019,OFFI_2019,TPCL_2019,HJI_2019
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2876,2877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2877,2878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2878,2879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2879,2880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
for csv in new_taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    csv = pd.read_csv(new_taz_se[0])
    csv['OFFI'] = csv['OFFI'] + csv['GVED'] + csv['HLTH']
    csv['TPCL'] = csv['OFFI'] + csv['RETEMP'] + csv['INDEMP']
    csv['HJI'] = csv['TOTHH']*1.8 + csv['TOTEMP']
    csv = csv[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
    csv.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
    
    base = base.merge(csv, on='TAZID', how='left')

In [38]:
base.shape

(2881, 258)

In [37]:
out_se_2019 = os.path.join(outputs, 'out_se_2019.shp')
base.spatial.to_featureclass(location=out_se_2019,sanitize_columns=False)

'e:\\Projects\\REMM-Process-Progression-Metrics-For-Web\\Outputs\\out_se_2019.shp'

In [43]:
identity = arcpy.Identity_analysis(out_se_2019, taz900, os.path.join(outputs, 'se_taz_900_identity.shp'))
identity2 = arcpy.Identity_analysis(out_se_2019, cities, os.path.join(outputs, 'se_cities_identity.shp'))
arcpy.management.DeleteField(identity,'TAZID', "KEEP_FIELDS")
arcpy.management.DeleteField(identity2,'CityArea', "KEEP_FIELDS")

<Result '.\\Outputs\\se_cities_identity.shp'>

['HH_2019',
 'POP_2019',
 'EMP_2019',
 'RTL_2019',
 'IND_2019',
 'OFFI_2019',
 'TPCL_2019',
 'HJI_2019',
 'HH_2020',
 'POP_2020',
 'EMP_2020',
 'RTL_2020',
 'IND_2020',
 'OFFI_2020',
 'TPCL_2020',
 'HJI_2020',
 'HH_2021',
 'POP_2021',
 'EMP_2021',
 'RTL_2021',
 'IND_2021',
 'OFFI_2021',
 'TPCL_2021',
 'HJI_2021',
 'HH_2022',
 'POP_2022',
 'EMP_2022',
 'RTL_2022',
 'IND_2022',
 'OFFI_2022',
 'TPCL_2022',
 'HJI_2022',
 'HH_2023',
 'POP_2023',
 'EMP_2023',
 'RTL_2023',
 'IND_2023',
 'OFFI_2023',
 'TPCL_2023',
 'HJI_2023',
 'HH_2024',
 'POP_2024',
 'EMP_2024',
 'RTL_2024',
 'IND_2024',
 'OFFI_2024',
 'TPCL_2024',
 'HJI_2024',
 'HH_2025',
 'POP_2025',
 'EMP_2025',
 'RTL_2025',
 'IND_2025',
 'OFFI_2025',
 'TPCL_2025',
 'HJI_2025',
 'HH_2026',
 'POP_2026',
 'EMP_2026',
 'RTL_2026',
 'IND_2026',
 'OFFI_2026',
 'TPCL_2026',
 'HJI_2026',
 'HH_2027',
 'POP_2027',
 'EMP_2027',
 'RTL_2027',
 'IND_2027',
 'OFFI_2027',
 'TPCL_2027',
 'HJI_2027',
 'HH_2028',
 'POP_2028',
 'EMP_2028',
 'RTL_2028',
 'IN

In [54]:
#apportion
app_columns = list(base.columns)[2:]
apportion = arcpy.ApportionPolygon_analysis(out_se_2019, app_columns, identity, os.path.join(outputs, 'new_se_apportion_to_taz900'), "AREA", "", "", "MAINTAIN_GEOMETRIES")

: 

: 